<a href="https://colab.research.google.com/github/ashwinamrutphale/aml-homeworks/blob/master/scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping UIUC CS faculty homepages

## System setup 

Before we start, make sure to install the required libraries
    
    pip install bs4
    pip install selenium

Since UIUC's website has some javascript rendered HTML content, we'll be using Selenium for scraping the content loaded dynamically by javascript. For this,you would also need to download a selenium supported browser webdriver.

e.g. For Chrome, download the appropriate webdriver from here: http://chromedriver.chromium.org/downloads, unzip it and save in current directory.

In [5]:
!pip install bs4
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [564 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [819 B]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/

In [0]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib

In [8]:
#create a webdriver object and set options for headless browsing
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


If you visit UIUC's CS Faculty Directory Listing: https://cs.illinois.edu/people/faculty/department-faculty , you'll notice that it has all the faculty listed there. 
Clicking on a faculty's image takes you to the Faculty Profile page. 
The profile page has more info about the faculty and sometimes contains the faculty hompage url under the Overview Tab > For More Information which is ultimately what we want to scrape.

Before we start scraping, we'll define some helper functions

In [0]:
#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,browser):
    browser.get(url)
    res_html = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

#tidies extracted text 
def process_bio(bio):
    bio = bio.encode('ascii',errors='ignore').decode('utf-8')       #removes non-ascii characters
    bio = re.sub('\s+',' ',bio)       #repalces repeated whitespace characters with single space
    return bio

''' More tidying
Sometimes the text extracted HTML webpage may contain javascript code and some style elements. 
This function removes script and style tags from HTML so that extracted text does not contain them.
'''
def remove_script(soup):
    for script in soup(["script", "style"]):
        script.decompose()
    return soup


#Checks if bio_url is a valid faculty homepage
def is_valid_homepage(bio_url,dir_url):
    try:
        #sometimes the homepage url points to the faculty profile page
        #which should be treated differently from an actual homepage
        ret_url = urllib.request.urlopen(bio_url).geturl() 
    except:
        return False       #unable to access bio_url
    urls = [re.sub('((https?://)|(www.))','',url) for url in [ret_url,dir_url]] #removes url scheme (https,http) or www 
    return not(urls[0]== urls[1])

We will now start scraping.

First, let's get links to all Faculty Profile pages by scraping the Directory Listing. You can use your browser's developer tools to find the required links within the HTML content. In Chrome, this can be done by right cliking on the webpage and choosing Inspect. Some basic knowledge of HTML and CSS would be required. After a bit of digging, you should notice that the link can be found under the <a\> tag under <div\> with class "photo nocaption" as shown:


![Screen%20Shot%202019-02-11%20at%2012.39.36%20PM.png](attachment:Screen%20Shot%202019-02-11%20at%2012.39.36%20PM.png)

Now we can specify exactly what needs to be extracted from the directory listing page

In [0]:
#extracts all Faculty Profile page urls from the Directory Listing Page
def scrape_dir_page(dir_url,browser):
    print ('-'*20,'Scraping directory page','-'*20)
    faculty_links = []
    faculty_base_url = 'https://biology.ucsd.edu'
    #execute js on webpage to load faculty listings on webpage and get ready to parse the loaded HTML 
    soup = get_js_soup(dir_url,browser)     
    for link_holder in soup.find_all('td',class_='name'): #get list of all <div> of class 'photo nocaption'
        #print(link_holder)
        try:
          rel_link = link_holder.find('a')['href'] #get url
          print(rel_link)
          #url returned is relative, so we need to add base url
          faculty_links.append(faculty_base_url+rel_link) 
        except Exception:
          pass  # o
        
    print ('-'*20,'Found {} faculty profile urls'.format(len(faculty_links)),'-'*20)
    return faculty_links

In [25]:
dir_url = 'https://biology.ucsd.edu/publicinfo/dwc?action=faculty_research_list' #url of directory listings of CS faculty
faculty_links = scrape_dir_page(dir_url,browser)
print(faculty_links)

-------------------- Scraping directory page --------------------
/research/faculty/sackerman
/research/faculty/oakbari
/research/faculty/eallen
/research/faculty/njallen
/research/faculty/carmour
/research/faculty/kasahina
/research/faculty/jsayres
/research/faculty/eazim
/research/faculty/tsb
/research/faculty/mbanghart
/research/faculty/adbarton
/research/faculty/e1bennett
/research/faculty/dberg
/research/faculty/ebier
/research/faculty/blbloodgood
/research/faculty/sbriggs
/research/faculty/sbrody
/research/faculty/wbusch
/research/faculty/mabutler
/research/faculty/hcang
/research/faculty/schalasani
/research/faculty/lchao
/research/faculty/lchilukuri
/research/faculty/adchisholm
/research/faculty/schoe
/research/faculty/jchory
/research/faculty/mchrispeels
/research/faculty/ecleland
/research/faculty/abcoleman
/research/faculty/hcookandersen
/research/faculty/j2cooke
/research/faculty/kcooper
/research/faculty/ncrawford
/research/faculty/mddaugherty
/research/faculty/midavid
/re

Now let's scrape the Faculty Profile pages to find faculty homepage urls. 

The faculty homepage urls are listed under the Overview tab > For more information on the Faculty profile page. So, again let's try to find where these urls are placed in the HTML content. 

You'll notice the Overview tab is the <div\> with id <overview\>. 

There could multiple headers (<h\>) under this tab e.g. Research etc., we're looking for the one with text "For more information".

There could be multiple listings (<li\>) under the header e.g. Reasearch Group Page, etc. There is no specific way to identify the one corresponding to the homepage. But, if you look at a few profile pages, you'll notice a few patterns which are generally found in the anchor text accompanying the homepage url. e.g. presence of substrings like 'page' or 'site' are good indicators as long as other words like 'group' or 'course' are not present. 

Ofcourse, there might still be a few cases where the patterns do not work. For such cases and for cases where the homepage url is actually not listed on the profile page, we will treat the faculty profile page as the homepage.


![Screen%20Shot%202019-02-11%20at%201.36.02%20PM.png](attachment:Screen%20Shot%202019-02-11%20at%201.36.02%20PM.png)



In [0]:
def scrape_faculty_page(fac_url,browser):

    soup = get_js_soup(fac_url,browser)
    homepage_found = False
    bio_url = ''
    bio = ''
    #find overview tab
    try:
        overview_tab = soup.find('div',id='bio')
    except:
        print (soup)
    #define patterns for the anchor text of homepage url
    faculty_last_name = soup.find('h1',id='facName').get_text().lower() #find faculty name
    homepage_txts = ['site','page',' '+faculty_last_name]
    exceptions = ['course ','research','group','cs','mirror','google scholar']
    #find the homepage url and extract all text from it
    for hdr in overview_tab.find_all('h2'):  #first find the required header
        if hdr.text.lower() == 'for more information':
            next_tag = hdr.find_next('li')
            #find <li> which has homepage url
            while next_tag is not None: 
                cand = next_tag.find('a')
                next_tag = next_tag.next_sibling    #sibling means element present at the same level
                cand_text = cand.get_text().lower()
                if (any(hp_txt in cand_text for hp_txt in homepage_txts) and 
                    not any(e in cand_text for e in exceptions)): #compare text to predefined patterns
                    bio_url = cand['href'] 
                    homepage_found = True
                    #check if homepage url is valid
                    if not(is_valid_homepage(bio_url,fac_url)): 
                        homepage_found = False
                    else:
                        try:
                            bio_soup = remove_script(get_js_soup(bio_url,browser)) 
                        except:
                            print ('Could not access {}'.format(bio_url))
                            homepage_found = False
                    break 
            if homepage_found:
                #get all the text from homepage(bio) since there's no easy to filter noise like navigation bar etc
                bio = process_bio(bio_soup.get_text(separator=' ')) 

    if not homepage_found:
        bio_url = fac_url #treat faculty profile page as homepage
        #we're only interested in some parts of the profile page namely the address
        #and information listed under the Overview, Research, Publication and Awards tab
        bio = soup.find('div',class_='dirprofile fullpage').get_text(separator=' ')+': '
        for tab in soup.find_all('div',class_='tab-pane'):
            bio += tab.get_text(separator=' ')+'. '
        bio = process_bio(bio)

    return bio_url,bio

In [13]:
#Scrape all faculty homepages using profile page urls
bio_urls, bios = [],[]
tot_urls = len(faculty_links)
for i,link in enumerate(faculty_links):
    print ('-'*20,'Scraping faculty url {}/{}'.format(i+1,tot_urls),'-'*20)
    bio_url,bio = scrape_faculty_page(link,browser)
    bio_urls.append(bio_url)
    bios.append(bio)

-------------------- Scraping faculty url 1/87 --------------------
-------------------- Scraping faculty url 2/87 --------------------
-------------------- Scraping faculty url 3/87 --------------------
-------------------- Scraping faculty url 4/87 --------------------
-------------------- Scraping faculty url 5/87 --------------------
-------------------- Scraping faculty url 6/87 --------------------
-------------------- Scraping faculty url 7/87 --------------------
-------------------- Scraping faculty url 8/87 --------------------
-------------------- Scraping faculty url 9/87 --------------------
-------------------- Scraping faculty url 10/87 --------------------
-------------------- Scraping faculty url 11/87 --------------------
-------------------- Scraping faculty url 12/87 --------------------
-------------------- Scraping faculty url 13/87 --------------------
-------------------- Scraping faculty url 14/87 --------------------
-------------------- Scraping faculty url 1

Finally, write urls and extracted bio to txt files

In [0]:
def write_lst(lst,file_):
  with open(file_,'w') as f:
    for l in lst:
        f.write(l)
        f.write('\n')

In [0]:
bio_urls_file = 'bio_urls.txt'
bios_file = 'bios.txt'
write_lst(bio_urls,bio_urls_file)
write_lst(bios,bios_file)